In [ ]:
import pennylane as qml
import pennylane.numpy as np

In [ ]:

def potential_energy_surface(symbols, bond_lengths):
    hf_energies = []
    _, params = qml.qchem.mol_basis_data('sto-3g', symbols)

    alpha = []

    for param in params:
      alpha.append(param[1])

    alpha = np.array(alpha, requires_grad = True)

    args = [alpha]

    for bond_length in bond_lengths:
      coordinates = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, bond_length]], requires_grad = False)
      mol = qml.qchem.Molecule(symbols, coordinates, alpha=alpha)
      hf_energies.append(qml.qchem.hf_energy(mol)(*args))

    return np.array(hf_energies)


def ground_energy(hf_energies):
    ind = np.argmin(hf_energies)
    return hf_energies[ind]

def reaction():
    molecules = {
        "H2":
            {"symbols": ["H", "H"], "E0": 0, "E_dissociation": 0, "bond lengths": np.arange(0.5, 9.3, 0.3)},
        "Li2":
            {"symbols": ["Li", "Li"], "E0": 0, "E_dissociation": 0, "bond lengths": np.arange(3.5, 8.3, 0.3)},
        "LiH":
            {"symbols": ["Li", "H"], "E0": 0, "E_dissociation": 0, "bond lengths": np.arange(2.0, 6.6, 0.3)}
    }


    for molecule in molecules.keys():
      hf_energies = potential_energy_surface(molecules[molecule]["symbols"], molecules[molecule]["bond lengths"])
      molecules[molecule]["E0"] = ground_energy(hf_energies)
      molecules[molecule]["E_dissociation"] = np.abs(molecules[molecule]["E0"] - hf_energies[-1])

    E_reactants = molecules["H2"]["E0"] + molecules["Li2"]["E0"]# the energy of the reactants
    E_activation = E_reactants + molecules["H2"]["E_dissociation"] + molecules["Li2"]["E_dissociation"]# the activation energy
    E_products = 2*molecules["LiH"]["E0"]# e, m = reaction()the energy of the products

    return np.array([E_reactants, E_activation, E_products])

In [ ]:
e, m = reaction()